In [1]:
import pandas as pd

Model Traning

In [2]:
df= pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [40]:
df= df.drop(labels=['id'],axis=1)

In [41]:
Y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [42]:
##Independent and dependent features
X=df.drop(labels=['price'],axis=1)
Y=df['price']

In [43]:
# Define which column should be ordinal-encoded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [44]:
#define the costom ranking for each ordinal variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [45]:
from sklearn.impute import SimpleImputer
##MEAN MEDIAN MOST FREQUENT
##for filling null values
##Handling missing values

In [46]:
from sklearn.preprocessing import StandardScaler
##Need to reach global minimum
## Handling Feature Scaling

In [47]:
from sklearn.preprocessing import OrdinalEncoder
#Ordinal encoding , mapping

In [48]:
from sklearn.pipeline import Pipeline
#pipelines

In [49]:
from sklearn.compose import ColumnTransformer

In [50]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
        
    ]

)

## Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('impuer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler()) ## After ordinalencoder we will get numerical values and them we have to scale it
    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [51]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30,random_state=33)

In [52]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [53]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.039814,-0.295907,-0.118919,-1.277045,-1.244180,-1.268243,0.874222,0.297443,1.35063
1,-0.996626,0.998091,0.402282,-1.141893,-1.198841,-1.065453,-0.131474,0.297443,0.68395
2,-0.910251,1.182948,0.402282,-1.060802,-1.062826,-0.964058,-2.142868,0.913192,-0.64941
3,0.471749,0.628377,-0.640119,0.597062,0.650964,0.672749,-1.137171,0.913192,-0.64941
4,-0.780689,0.905663,-0.118919,-0.826538,-0.799864,-0.746783,0.874222,-0.318306,1.35063


In [54]:
## Model Traing
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [55]:
regression=LinearRegression()
regression.fit(X_train,Y_train)

LinearRegression()

In [56]:
regression.coef_

array([ 6417.53881888,  -130.2755795 ,   -66.5435773 , -1767.6050283 ,
        -406.61670675,   -84.23786515,    71.64415167,  -467.49658585,
         651.42052431])

In [57]:
regression.intercept_

3977.8208278905704

In [58]:
import numpy as np
def evalute_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [59]:
## Train multiple Model

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]


for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)
    
    ##Make prediction
    Y_pred=model.predict(X_test)
    mae,rmse,r2_square=evalute_model(Y_test,Y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n') 

LinearRegression
Model Training Performance
RMSE: 1003.9683231417787
MAE: 673.4979988108095
R2 score 93.75571899255428


Lasso
Model Training Performance
RMSE: 1004.1774865202909
MAE: 674.7072899734013
R2 score 93.75311689658488


Ridge
Model Training Performance
RMSE: 1003.9723371874096
MAE: 673.5296527449725
R2 score 93.75566906094105


ElasticNet
Model Training Performance
RMSE: 1526.4076959180638
MAE: 1060.7705550659127
R2 score 85.56610596968004




In [60]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']